In [ ]:
from PIL import Image
import numpy as np
from train_dataset import img_patch
import os
from sklearn.model_selection import train_test_split


# Load and preprocess the images
datapath = 'dataset/mini_testdatensatz'
datapath = datapath + '/train'
im_size= [192,192]


images= []

filenames_train = os.listdir(datapath)


for filename in filenames_train:
    img = Image.open(os.path.join(datapath,filename)) # Convert to grayscale
    img_resized = img_patch(img, im_size)
    img_grey = img_resized.convert('L')  # Resize the image
    img_numpy = np.array(img_grey) / 255.0             # Normalize the pixel values
    #img_reshape = img_numpy.reshape((im_size[0], im_size[1], 1))  # Add the channel dimension
    images.append(img_numpy)



# Convert the list of images to a NumPy array
images = np.array(images)
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)



In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# Define the input shape
input_img = Input(shape=(im_size[0], im_size[1], 1))

# Define the encoder layers
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Define the decoder layers
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(input_img, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder model
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test, x_test))

# Use the trained autoencoder to segment the input image
segmented_img = autoencoder.predict(input_img)


In [ ]:
from PIL import Image
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# Define a function to load and preprocess the images
def preprocess_images(datapath, im_size):
    images = []
    filenames_train = os.listdir(datapath)

    for filename in filenames_train:
        img = Image.open(os.path.join(datapath, filename)).convert('L')  # Convert to grayscale
        img_resized = img.resize(im_size)  # Resize the image
        img_numpy = np.array(img_resized, dtype='float32') / 255.0  # Normalize the pixel values
        img_reshape = img_numpy.reshape((*im_size, 1))  # Add the channel dimension
        images.append(img_reshape)

    # Convert the list of images to a NumPy array
    images = np.array(images)
    return images

# Load and preprocess the training and testing images
datapath = 'dataset/mini_testdatensatz/train'
im_size = [192, 192]
x_train = preprocess_images(datapath, im_size)
datapath = 'dataset/mini_testdatensatz/test'
x_test = preprocess_images(datapath, im_size)

# Define the input shape
input_img = Input(shape=(*im_size, 1))

# Define the encoder layers
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Define the decoder layers
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(input_img, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder model
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test, x_test))


In [ ]:
# Use the trained autoencoder to segment a new image
new_image_path = 'path/to/new/image.jpg'
new_img = Image.open(new_image_path).convert('L')
new_img_resized = new_img.resize(im_size)
new_img_numpy = np.array(new_img_resized, dtype='float32') / 255.0
new_img_reshape = new_img_numpy.reshape((*im_size, 1))
segmented_img = autoencoder.predict(np.array([new_img_reshape]))